In [14]:
import os
import glob
import pandas as pd
import numpy as np


In [6]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/smpd3_dataset/MigrationResults'

In [12]:
# Add analysis date here to apply to dataframe
analysis_date = '20181103'

path = os.path.abspath('')+'/CSVs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
#    df['Date'] = expt_date
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in ROI label and Image name columns
    df['delete1'], df['ROI'], df['delete2'], df['delete3'] = zip(*df['Label'].map(lambda x: x.split(':')))
    df['ExptDate'], df['Treatment'], df['Stains'], df['Embryo'], df['Somites'], df['ImageMag'] = zip(*df['Image'].map(lambda x: x.split('_')))
    list_.append(df)

full_df = pd.concat(list_)
full_df = full_df.drop(['Label', 'Mean', 'IntDen', 'RawIntDen', 'delete1', 'delete2', 'delete3'], axis=1)
full_df.head(10)

,,Area,Image,ROI,ExptDate,Treatment,Stains,Embryo,Somites,ImageMag
0,1,71843.804,20180418_SMPD3gRNA1DNA_Pax7NcadLaminin_Emb1_8s...,CntlArea,20180418,SMPD3gRNA1DNA,Pax7NcadLaminin,Emb1,8ss,10x
1,2,51341.649,20180418_SMPD3gRNA1DNA_Pax7NcadLaminin_Emb1_8s...,ExptArea,20180418,SMPD3gRNA1DNA,Pax7NcadLaminin,Emb1,8ss,10x
0,1,80357.839,20171016_SMPD3gRNA1DNA_Pax7Sox9_Emb5_9ss_10x,CntlArea,20171016,SMPD3gRNA1DNA,Pax7Sox9,Emb5,9ss,10x
1,2,88925.670,20171016_SMPD3gRNA1DNA_Pax7Sox9_Emb5_9ss_10x,ExptArea,20171016,SMPD3gRNA1DNA,Pax7Sox9,Emb5,9ss,10x
0,1,88178.692,"20180630_SMPD3MO0,8mM_Pax7_Emb4_8ss_10x",CntlArea,20180630,"SMPD3MO0,8mM",Pax7,Emb4,8ss,10x
1,2,62290.316,"20180630_SMPD3MO0,8mM_Pax7_Emb4_8ss_10x",ExptArea,20180630,"SMPD3MO0,8mM",Pax7,Emb4,8ss,10x
0,1,97302.020,"20180514_SMPD3MO0,8mM_Pax7Laminin_Emb13_8ss_10x",CntlArea,20180514,"SMPD3MO0,8mM",Pax7Laminin,Emb13,8ss,10x
1,2,40681.740,"20180514_SMPD3MO0,8mM_Pax7Laminin_Emb13_8ss_10x",ExptArea,20180514,"SMPD3MO0,8mM",Pax7Laminin,Emb13,8ss,10x
0,1,74199.298,"20180514_SMPD3MO0,8mM_Pax7Laminin_Emb8_7ss_10x",CntlArea,20180514,"SMPD3MO0,8mM",Pax7Laminin,Emb8,7ss,10x
1,2,44131.504,"20180514_SMPD3MO0,8mM_Pax7Laminin_Emb8_7ss_10x",ExptArea,20180514,"SMPD3MO0,8mM",Pax7Laminin,Emb8,7ss,10x


In [4]:
df_byTreatment = full_df.groupby(['Treatment', 'Image', 'ROI'])['Image', 'ROI', 'Area', 'Treatment'].mean()
df_treatment = pd.DataFrame(df_byTreatment.xs(treatment))
area_cntl = df_byTreatment.xs('CntlArea', level='ROI')
area_cntl.head()


Area
Treatment    Image                                                      
SMPD3MO0,8mM 20180514_SMPD3MO0,8mM_Pax7Laminin_Emb12_8ss_10x  116506.057
             20180514_SMPD3MO0,8mM_Pax7Laminin_Emb13_8ss_10x   97302.020
             20180514_SMPD3MO0,8mM_Pax7Laminin_Emb1_8ss_10x   129671.718
             20180514_SMPD3MO0,8mM_Pax7Laminin_Emb3_9ss_10x   173182.400
             20180514_SMPD3MO0,8mM_Pax7Laminin_Emb6_8ss_10x    72508.322

# For Outlier detection using Inter Quartile Range
import numpy as np

def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))

In [192]:
# Get a list of treatments to iterate through
treatment_list = full_df.Treatment.unique()
treatment_list = treatment_list.tolist()

# Group dataframe by treatment
df_byTreatment = full_df.groupby(['Treatment', 'Image', 'ROI'])['Image', 'ROI', 'Area', 'Treatment'].mean()

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
for i in treatment_list:
    # Slice dataframe to process only embryos with given treatment
    treatment = i
    df_treatment = pd.DataFrame(df_byTreatment.xs(treatment))

    # Extract Cntl and Expt areas
    area_cntl = df_treatment.xs('CntlArea', level='ROI')
    area_expt = df_treatment.xs('ExptArea', level='ROI')

    # Generate Expt/Cntl Area Ratios
    area_ratios = pd.DataFrame(area_expt / area_cntl)
    area_ratios.columns = ['Expt/Cntl Area']
    
    # Calculate if any outliers exist in area_ratios using interquartile range
    quartile_1, quartile_3 = np.percentile(area_ratios['Expt/Cntl Area'], [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outlier = np.where((area_ratios['Expt/Cntl Area'] > upper_bound) | (area_ratios['Expt/Cntl Area'] < lower_bound))
    outlier_image = area_ratios.iloc[outlier].reset_index()
    outlier_image = outlier_image['Image'].tolist()
    
    if len(outlier) > 0:
        area_cntl_noOutlier = area_cntl.loc[~(area_cntl['Image'] == outlier_image[0])]
        area_expt_noOutlier = area_expt.loc[~(area_expt['Image'] == outlier_image[0])]
        area_ratios_noOutlier = area_ratios.loc[~(area_ratios['Image'] == outlier_image[0])]
        
        # Normalize all migration area values to mean of control group
        norm_cntl_noOutlier = area_cntl_noOutlier/(float(area_cntl.mean()))
        norm_cntl_noOutlier.columns = ['Norm Cntl Area']
        norm_expt_noOutlier = area_expt_noOutlier/(float(area_cntl.mean()))
        norm_expt_noOutlier.columns = ['Norm Expt Area']
        norm_areas_noOutlier = pd.concat([norm_cntl_noOutlier, norm_expt_noOutlier], axis=1, sort=False)
        
        # Combine processed values into single dataframe and output as csv file analysis date + 'MigrationResults.csv'
        area_cntl_noOutlier = pd.DataFrame(area_cntl_noOutlier)
        area_cntl_noOutlier.columns = ['Cntl Area']
        area_expt_noOutlier = pd.DataFrame(area_expt_noOutlier)
        area_expt_noOUtlier.columns = ['Expt Area']
        results = pd.concat([area_cntl_noOutlier, area_expt_noOutlier, area_ratios_noOutlier,
                            norm_cntl_noOutlier, norm_expt_noOutlier], axis=1, sort=True)
        results.to_csv(analysis_date + '_' + treatment + '_MigrationResults.csv')
        
    else:
        # Normalize all migration area values to mean of control group
        norm_cntl = area_cntl/(float(area_cntl.mean()))
        norm_cntl.columns = ['Norm Cntl Area']
        norm_expt = area_expt/(float(area_cntl.mean()))
        norm_expt.columns = ['Norm Expt Area']
        norm_areas = pd.concat([norm_cntl, norm_expt], axis=1, sort=False)

        # Combine processed values into single dataframe and output as csv file analysis date + 'MigrationResults.csv'
        area_cntl = pd.DataFrame(area_cntl)
        area_cntl.columns = ['Cntl Area']
        area_expt = pd.DataFrame(area_expt)
        area_expt.columns = ['Expt Area']
        results = pd.concat([area_cntl, area_expt, area_ratios, norm_cntl, norm_expt], axis=1, sort=True)
        results.to_csv(analysis_date + '_' + treatment + '_MigrationResults.csv')